In [1]:
import stock
from importlib import reload
reload(stock)

import pandas
import time
import os

## Load list of stocks from files

### Load lists of stocks from Saint-Petersburg stock exchange

In [2]:
stocks1 = pandas.read_csv('data/stocks.spb', sep=';', encoding='cp1251', index_col=6)
stocks1 = pandas.DataFrame(index = stocks1.index, data = {'name': stocks1['e_full_name']})
stocks1.index.name = 'symbol'
stocks1[:5]

,name
symbol,
SOHU,The Bank of New York Mellon
ALFAperp0222,Alfa Bond Issuance PLC
PRSP,Perspecta Inc.
WH,"Wyndham Hotels & Resorts, Inc."
APY,Apergy Corporation


### Load lists of stocks from TCS

In [3]:
stocks2 = pandas.read_csv('data/stocks.tcs', index_col=0)
stocks2 = pandas.DataFrame(index = stocks2.index, data = {'name': stocks2['Name']})
stocks2.index.name = 'symbol'
print('Loaded list of %i stocks' % len(stocks2))
stocks2[-5:]

Loaded list of 517 stocks


,name
symbol,
ZNH,China Southern Airlines Company Limited
ZTS,Zoetis Inc.
V,Visa
NDAQ,Nasdaq
AAPL,Apple


### Unite stocks lists

In [4]:
stocks = pandas.concat([stocks1, stocks2])
stocks = stocks.reset_index().drop_duplicates(subset='symbol').set_index('symbol')
stocks = stocks.sort_index()
print('Totaly: %i stocks' % len(stocks))
stocks[:5]

Totaly: 656 stocks


,name
symbol,
A,"Agilent Technologies, Inc."
AA,Alcoa Corporation
AABA,Altaba Inc.
AAL,American Airlines Group Inc.
AAP,"Advance Auto Parts, Inc."


### Apply blacklist

In [5]:
blacklist = [*map(lambda s: s.strip(), open('data/blacklist').readlines())]
prev_size = len(stocks)
stocks = stocks.drop(blacklist, errors='ignore')
print('Totaly: %i stocks, %i were deleted' % (len(stocks), prev_size - len(stocks)))

Totaly: 656 stocks, 0 were deleted


### Apply whitelist

In [40]:
shortlist = []
shortlist = [*map(lambda s: s.strip(), open('data/shortlist').readlines())]
if shortlist:
    prev_size = len(stocks)
    stocks = stocks.loc[shortlist]
    print('Totaly: %i stocks, %i were deleted' % (len(stocks), prev_size - len(stocks)))
    stocks[:5]

Totaly: 12 stocks, 644 were deleted


## Download prices history

In [41]:
history_type = 'daily'
# history_type = 'monthly'

In [42]:
av = stock.AlphaVantage()
prices = {}
i = 0
total = len(stocks.index)
for symbol in stocks.index:
    i += 1
    try:
        print('[%i/%i] Loading symbol "%s" ...' % (i, total, symbol))
        if history_type == 'monthly':
            prices_history = av.monthly_adjusted(symbol=symbol)
        elif history_type == 'daily':
            prices_history = av.daily_adjusted(symbol=symbol)
        else:
            raise Exception('Unknown history type: {}'.format(history_type))
        res = 'OK' if stock.ok(prices_history) else (prices_history if type('') == type(prices_history) else 'FAIL')
        print('[%i/%i] Loading symbol "%s" - "%s"' % (i, total, symbol, res))
    except Exception as exc:
        prices_history = None
        print('[%i/%i] Symbol "%s" exception: %r' % (i, total, symbol, exc))
    prices[symbol] = prices_history
    if i < total:
        time.sleep(4)

[1/12] Loading symbol "ADBE" ...
[1/12] Loading symbol "ADBE" - "OK"
[2/12] Loading symbol "RACE" ...
[2/12] Loading symbol "RACE" - "OK"
[3/12] Loading symbol "INTU" ...
[3/12] Loading symbol "INTU" - "OK"
[4/12] Loading symbol "MA" ...
[4/12] Loading symbol "MA" - "OK"
[5/12] Loading symbol "NDAQ" ...
[5/12] Loading symbol "NDAQ" - "OK"
[6/12] Loading symbol "NFLX" ...
[6/12] Loading symbol "NFLX" - "OK"
[7/12] Loading symbol "RHT" ...
[7/12] Loading symbol "RHT" - "OK"
[8/12] Loading symbol "CRM" ...
[8/12] Loading symbol "CRM" - "OK"
[9/12] Loading symbol "TROW" ...
[9/12] Loading symbol "TROW" - "FAIL"
[10/12] Loading symbol "TXN" ...
[10/12] Loading symbol "TXN" - "FAIL"
[11/12] Loading symbol "UNH" ...
[11/12] Loading symbol "UNH" - "FAIL"
[12/12] Loading symbol "V" ...
[12/12] Loading symbol "V" - "FAIL"


## Try to download failed positions again

In [45]:
i = 0
total = len(prices)
for (symbol, res) in prices.items():
    i += 1
    if stock.ok(res):
        print('[%i/%i] Symbol "%s" - "OK"' % (i, total, symbol))
        continue
    try:
        print('[%i/%i] Loading symbol "%s" ("%s") ...' % (i, total, symbol, stocks['name'].loc[symbol]))
        if history_type == 'monthly':
            prices_history = av.monthly_adjusted(symbol=symbol)
        elif history_type == 'daily':
            prices_history = av.daily_adjusted(symbol=symbol)
        else:
            raise Exception('Unknown history type: {}'.format(history_type))
        res = 'OK' if stock.ok(prices_history) else (prices_history if type('') == type(prices_history) else 'FAIL')
        print('[%i/%i] Loading symbol "%s" ("%s") - "%s"' % (i, total, symbol, stocks['name'].loc[symbol], res))
    except Exception as exc:
        prices_history = None
        print('[%i/%i] Symbol "%s" exception: %r' % (i, total, symbol, exc))
    prices[symbol] = prices_history
    if i < total:
        time.sleep(5)

[1/12] Symbol "ADBE" - "OK"
[2/12] Symbol "RACE" - "OK"
[3/12] Symbol "INTU" - "OK"
[4/12] Symbol "MA" - "OK"
[5/12] Symbol "NDAQ" - "OK"
[6/12] Symbol "NFLX" - "OK"
[7/12] Symbol "RHT" - "OK"
[8/12] Symbol "CRM" - "OK"
[9/12] Symbol "TROW" - "OK"
[10/12] Symbol "TXN" - "OK"
[11/12] Symbol "UNH" - "OK"
[12/12] Symbol "V" - "OK"


### Remove invalid symbols

In [46]:
prices.pop('LLY.TI', None)
prices.pop('Symbol', None)
prices.pop('VNE$', None)
prices.pop('ZB^A', None)

### List symbols with invalid prices history

In [47]:
invalid_prices = dict([*filter(lambda item: not item[1], prices.items())])
prices = dict([*filter(lambda item: item[1], prices.items())])
print('Invalid prices for symbols: %s' % ', '.join(invalid_prices.keys()))

Invalid prices for symbols: 


## Represet prices history as list of DataFrames

In [48]:
def month_prices_to_dataframe(symbol, prices):
    d = [(symbol, date, price.open, price.close, price.volume, price.high, price.low) for (date, price) in prices.items()]
    return pandas.core.frame.DataFrame(d, columns=['symbol', 'date', 'open', 'close', 'volume', 'high', 'low'])
    

In [49]:
prices_dataframes = []
for (symbol, month_prices) in prices.items():
    df = None
    try:
        df = month_prices_to_dataframe(symbol, month_prices)
        prices_dataframes.append(df)
    except Exception as exc:
        print('DataFrame for symbol "%s" - FAIL' % symbol)
        

## Save DataFrames with symbols prices history to "./hostory" directory

In [50]:
def save_dataframe(dir_path, df):
    path = os.path.join(dir_path, df['symbol'][0] + '.csv')
    df.to_csv(path)


In [51]:
if history_type == 'monthly':
    history_path = 'history'
elif history_type == 'daily':
    history_path = 'history_daily'
else:
    raise Exception('Unknown history type: {}'.format(history_type))
for df in prices_dataframes:
    save_dataframe(history_path, df)

### Test saved data

In [52]:
pandas.read_csv(history_path + '/A.csv', index_col=0)

,symbol,date,open,close,volume,high,low
0,A,2018-07-18,63.030000,62.9900,1202284,63.230000,62.750000
1,A,2018-07-17,62.520000,63.0400,1469834,63.260000,62.150000
2,A,2018-07-16,63.230000,62.7200,929264,63.230000,62.630000
3,A,2018-07-13,63.520000,63.3800,963652,63.830000,63.210000
4,A,2018-07-12,63.330000,63.4500,1748125,63.780000,63.290000
5,A,2018-07-11,62.590000,62.8600,1780485,63.280000,62.590000
6,A,2018-07-10,63.370000,63.6700,1924366,63.740000,63.070000
7,A,2018-07-09,62.430000,63.1200,1508550,63.315000,62.430000
8,A,2018-07-06,61.920000,62.2900,2269660,62.400000,61.650000
9,A,2018-07-05,61.480000,61.8000,2247532,61.890000,60.910000
